## **Proyecto final modelo de predicción del servicio de taxis para usuarios recurrentes**

Utilizando el proyecto de tesis

1) Exploración, análisis, selección de variables y creación del
modelo sobre conjunto de train (12 puntos)

2) Creación de
pipeline (10 puntos)

3) Evaluación de
modelos usando validación cruzada (incluyendo pipeline) (5 puntos)

4) Ajuste de
hiperparámetros de los modelos seleccionados (5 puntos)

5) Creación de
Joblib

6) Despliegue del
modelo usando Sagemaker SDK. Creación de Endpoint (8 puntos)

7) Creación de un
servicio web que reciba los datos del usuario por POST desde un
formulario, solicite al endpoint de sagemaker la predicción del
modelo y devuelva la respuesta al usuario final como un HTML.(Usar
un EC2 con ip pública) (10 puntos)

---------------------------------------------------------------------

1) Exploración, análisis, selección de variables y creación del modelo sobre conjunto de train

In [145]:
#Importación de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from numpy import NaN
from datetime import date, datetime, timedelta
from os import name

In [146]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import plot_confusion_matrix
from matplotlib.colors import ListedColormap
#from mlxtend.plotting import plot_decision_regions
#modelos
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [147]:
NUM_OF_WEEK = 8.0
HOURS_OF_DAY = 24
DAYS_OF_WEEK = 7
FORMAT_DATE = '%Y-%m-%d'
DATE_START_TRAINIG = pd.to_datetime("2022-04-27", format=FORMAT_DATE)
DATA_FILE = "s3://proyecto-taxis-usa/Data/SERVICIO_UNIFICADO_2022.parquet.gzip"

In [ ]:
#Se lee el archivo de un parquet a un dataframe
df_taxis = pd.read_parquet(DATA_FILE) 
#Se visualizan los datos
df_taxis.head(5)

In [ ]:
df_taxis.info()

In [ ]:
#Se cuentan cuantos datos se encuentran nulos
df_taxis.isnull().sum()

In [ ]:
#Se filtran los estados importantes 
df_taxis = df_taxis[(df_taxis["ESTADO"]=="ABANDONADO") | (df_taxis["ESTADO"]=="CUMPLIDO") | (df_taxis["ESTADO"]=="CANCELADO") | (df_taxis["ESTADO"]=="FINALIZADO")]
service_date = pd.to_datetime(df_taxis['FECHACOMPLETA'], format='%Y/%m/%d %H:%M:%S')
#Se crea un nuevo campo con el mes
df_taxis['MES'] = service_date.dt.month
#Se toman los meses de enero a mayo del 2022
df_taxis = df_taxis[(df_taxis["MES"]<=5)]
#Se crea el campo año y mes
df_taxis['FECHA_MES'] = service_date.dt.strftime('%Y-%m')
#Se crea un nuevo campo con la fecha en formato YYYY/MM/DD
df_taxis['FECHA'] = service_date.dt.strftime(FORMAT_DATE)

## Visualización de datos iniciales

In [ ]:
#Se visualiza la cantidad de servicios por estado
plt.figure(figsize=(10,8))
sns.histplot(df_taxis['ESTADO'])
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
#Se visualiza la cantidad por origen
plt.figure(figsize=(10,8))
sns.histplot(df_taxis['ORIGEN'])
plt.xticks(rotation='vertical')
plt.show()

Se visualizan los datos del dataFrame en un histograma

In [ ]:
#Se require presentar gráfica de histograma por cada columna o datos de dataframe
df_taxis.hist(figsize=(20,10))
plt.show()

Se visualizan las corelaciones entre variables

In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(df_taxis.corr(method='pearson'),annot=True)
plt.show()

Se visualiza un histograma con la cantidad de servicios por estado

In [ ]:
df_taxis.groupby(['FECHA_MES','ESTADO'])['ID'].count().unstack('ESTADO').plot(kind='bar',stacked=True, fontsize=14, xlabel="Mes", ylabel="Cantidad servicios", figsize=(16,8), title="Servicios por periodo", rot=0)
plt.show()

Se eliminan los datos que no se necesitan para predecir el modelo

In [ ]:
df_taxis.head(5)

In [ ]:
#Eliminamos las columan que no son útiles para el modelo
df_taxis = df_taxis.drop(columns=["IDCIUDAD","USUARIO","ID","LATITUDDESTINO","LONGITUDDESTINO","LATITUD","LONGITUD","FECHACOMPLETA","ORIGEN","ESTADO","FECHA_MES"])
#Se eliminan los valores duplicados del dataframe y se ordenan por los campos más relevangtes
df_taxis = df_taxis.drop_duplicates().sort_values(by=['TELEFONOORIGEN','FECHA','DIADESEMANA','HORA'])
#Se reindexa nuevamente el dataFrame
df_taxis = df_taxis.reset_index(drop=True)
#Se visualizan como queda el dataFrame
df_taxis.head(5)

Se contabilizan cuantos servicios tiene asignado un teléfono después de eliminar los duplicados

In [ ]:
count_data = df_taxis.loc[:,['TELEFONOORIGEN']].value_counts()
count_data

Se visualiza la distribución de los datos

In [ ]:
#Se visualizan los valores de los datos
count_data.describe().T

## Se calcula el outlayer de los datos

In [ ]:
#Se calcula el quartil 25
q25 = count_data.quantile(0.25)
#Se calcula el quartil 75
q75 = count_data.quantile(0.75)
#Se calcula el rango inter quartil
iqr = q75 - q25

print(f"El porcentaje del quartil 25: {q25}")
print(f"El porcentaje del quartil 75: {q75}")
print(f"El rango inter quartil es: {iqr}")

Se calcula el umbral de los valores atípicos

In [ ]:
#Se calcula el umbral de los  valores atípicos
outliers_threshold = ( q75 + 3 * iqr )
print(f"El umbral de los valores atípicos es: {outliers_threshold}")
print(f"La cantidad de valores atípicos son: {sum(count_data > outliers_threshold) }")

Eliminamos los datos que esten por debajo del umbral de los valores atípicos

In [ ]:
df_cleaned = df_taxis.copy()
#Se descartan los que tienen menos del umbral de los valores atípicos
df_cleaned = df_cleaned.groupby('TELEFONOORIGEN').filter(lambda x: x['TELEFONOORIGEN'].count() > outliers_threshold)
#Se visualizan los datos agrupados por mes para determinar cuantos servicios tomó un usuario
df_cleaned.groupby(['MES','TELEFONOORIGEN'])['TELEFONOORIGEN'].count()

In [ ]:
#Se visualizan los valores de los datos
df_cleaned.loc[:,['TELEFONOORIGEN']].value_counts().describe()

## Visualización de datos final

In [ ]:
#Se presenta un gráfico general de los datos
sns.pairplot(df_cleaned)
plt.show()

In [ ]:
#Se obtiene la matrix de correlación con los datos finales
corr_matrix = df_cleaned.corr()
corr_matrix

#se presenta gráfico  de correlación
f, ax = plt.subplots(figsize=(16, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr_matrix, annot=True, mask = mask, cmap=cmap )
plt.title('Relacion entre variables del dataframe',fontsize=20,fontname='serif')
plt.ylabel('Y', color='green', fontsize=25)
plt.xlabel('X', color='Red', fontsize=25)

In [ ]:
#Se realiza un conteo de la cantidad de datos por usuario
df_cleaned.loc[:,['TELEFONOORIGEN']].value_counts()

Se filtra un usuario con datos

In [ ]:
#Se crea un nuevo dataframe con los teléfonos para entrenar cada usuario personalizado
df_phone = df_cleaned.copy()

'''
df_phone = df_phone[(df_phone["TELEFONOORIGEN"]=="3115590650") | 
(df_phone["TELEFONOORIGEN"]=="3107584482")| 
(df_phone["TELEFONOORIGEN"]=="3118082696")| 
(df_phone["TELEFONOORIGEN"]=="3152175765")]
'''
df_phone = df_phone[(df_phone["TELEFONOORIGEN"]=="3107584482")]
df_phone = df_phone.drop(columns=["TELEFONOORIGEN"])

## Se visualiza la candidad de servicios por semana durantes los meses del usuario

In [ ]:
df_phone.groupby(['MES','DIADESEMANA'])['HORA'].count().unstack('DIADESEMANA').plot(kind='bar',stacked=True, fontsize=14, xlabel="Mes", ylabel="Cantidad servicios", figsize=(16,8), title="Servicios por periodo", rot=0)
plt.show()

In [ ]:
#Se agrupa por 'Mes' y 'Día de la semana'
df_phone.groupby(['MES','DIADESEMANA'])['HORA'].count().unstack('DIADESEMANA').plot(kind='bar', xlabel="Mes", ylabel="Cantidad servicios", figsize=(16,8), title="Servicios por periodo", rot=0)
plt.show()

Se crea la función que rellena con datos sin servicios

In [ ]:
#Funcion que permite llenar los servicios tomados con 1 y los no tomados con 0
def setDataService(data, from_date, to_date):
  remaining_days = to_date.date() - from_date.date()
  service_array = []
  for i in range(remaining_days.days+1):
    last_time = from_date + timedelta(days=i)
    if last_time.date().isoweekday() == to_date.date().isoweekday():
      for hour in range(HOURS_OF_DAY):
        #Se filtra por el mismo día de la semana, fecha y hora
        print(f"{last_time.isoweekday()} - {last_time.strftime(FORMAT_DATE)} - {hour}")
        df_service = data[(data['DIADESEMANA'] == last_time.isoweekday()) & (data['FECHA'] == last_time.strftime(FORMAT_DATE).date()) & (data['HORA'] == hour) ].to_numpy()
        if len(df_service) > 0:
          #Se llena con valor en 1 porqué se encontró un servicio
          print(np.append(df_service, 1))
          service_array.append(np.append(df_service, 1))
        else:
          #Se llena con valor en 0 porqué no se encontró ningún servicio
          #print([last_time.isoweekday(),hour,last_time.month,last_time.date().strftime(FORMAT_DATE),0])
          service_array.append([last_time.isoweekday(),hour,last_time.month,last_time.date().strftime(FORMAT_DATE),0])
  return pd.DataFrame(service_array, 
             columns=np.append(np.array(data.columns.tolist()), 'SERVICIO'))

In [ ]:
df_phone.tail(5)

In [ ]:
df_phone_v2 = setDataService(df_phone, pd.to_datetime("2022-01-01", format=FORMAT_DATE) , pd.to_datetime("2022-05-29", format=FORMAT_DATE) )

In [ ]:
df_phone_v2

In [ ]:
df_phone_v2['SERVICIO'].value_counts()